In [1]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import time


import grpc
import daq_data_pb2
import daq_data_pb2_grpc

from daq_data_client import (
    reflect_services,
    unpack_pano_image,
    format_capture_science_response,
    make_rich_logger
)

logger = make_rich_logger(__name__)

In [ ]:
# host = "localhost"
host = "10.0.0.60"
port = 50051

connection_target = f"{host}:{port}"
with grpc.insecure_channel(connection_target) as channel:
    reflect_services(channel)
    stub = daq_data_pb2_grpc.DaqDataStub(channel)

    capture_science_request = daq_data_pb2.CaptureScienceRequest(
        stream_movie_data=True,
        stream_pulse_height_data=True,
    )
    try:
        capture_science_responses = stub.CaptureScience(capture_science_request)
        for capture_science_response in capture_science_responses:
            header, img = unpack_pano_image(capture_science_response.pano_image)

            plt.imshow(img)
            clear_output(wait=False)
            logger.info(format_capture_science_response(capture_science_response))
            display(plt.gcf())
    except KeyboardInterrupt:
        logger.info(f"'^C' received, closing connection to DaqData server at {repr(connection_target)}")
    except grpc.RpcError as rpc_error:
        logger.error(f"{type(rpc_error)}\n{repr(rpc_error)}")
    finally:
        capture_science_responses.cancel()
        plt.close()


13:29:38 INFO     [tid=8757206784] [<module>()] CaptureScienceResponse: name='test_movie_data',    ]8;id=240125;file:///var/folders/5q/9jqcz6pj3053zxqf_d22qb680000gr/T/ipykernel_30905/4243170462.py\4243170462.py]8;;\:]8;id=574780;file:///var/folders/5q/9jqcz6pj3053zxqf_d22qb680000gr/T/ipykernel_30905/4243170462.py#21\21]8;;\
                  message='testing', timestamp='2025-07-07T20:29:38.631056', resp_type='DATA',                     
                  header={'test1': 1.0, 'test0': 0.0}                                                              